In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [2]:
import spacy
import pandas as pd
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

from util import remove_fullspace_and_newline, reweight_distribution

C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [3]:
# 歌詞データの読み込み
df = pd.read_csv('./datasets/all_songs.csv')
lyrics = df['Lyric']
lyrics.head()

0    まるで荒れる波涛のように背筋つらぬき　心狂わす出逢いそう　出逢い夢うつつと見紛(みまご)うほ...
1    Don't Don't Don't Stay Good-byeDon't Don't Don...
2    YES♪広い空のようなみんな夢を見てるそして叶えてく輝くこの宇宙(そら)でどこ行(ゆ)こうM...
3    いっしょに歌おういっしょの時代(とき)の中いっしょで行こういっしょな夢を見よういっしょにいよ...
4    アッ！とね　言わせて見たいいっぱい愛があふれるウットリするような世界創ろうガッカリするのまだ...
Name: Lyric, dtype: object

In [19]:
# 前処理(全角スペースや改行を除去)
removed_list = [remove_fullspace_and_newline(lyric) for lyric in lyrics]
removed_list[-2]

'瑠璃色金魚は恋い焦がれる凛と咲き誇る花菖蒲吐き出す空気は泡の模様決してあなたの心に届かないのはなびらひらひらと水面に落ちて震える指先時間が止まるわ目が覚めた余韻の余白外の世界はねえなんて眩しい嘘だとしても罪深過ぎたの眩暈がしても心地いいのはもう求めてるから瑠璃色金魚が見上げるのは凛と佇んだ花菖蒲私あなたのようになれたらもっと上手く微笑えますか灯した明かりは燃えないまま今も青く棚引いている曇った硝子を溶かすほどの秘密もしかして私持ってますか雨は空に落ち愛すれば消えるものと思ってた鏡の世界に逆さまに映った好奇心湧き上がる思いを掬い上げては砂糖漬けにしてまた飲み込むのあなたにいつか味見してほしいと夢を見ながら瑠璃色金魚が知らないのは強く根を張った花菖蒲目の前に見えるもの全てが現実ってことはないのあの時触れてくれた温もり光失くしては枯れていく悲しみで泣く私の涙また毒になってしまう抜け出したい瑠璃色金魚が見上げるのは凛と佇んだ花菖蒲私あなたのようになれたらもっと美しく咲き誇れますか瑠璃色金魚が知らないのは強く根を張った花菖蒲目の前に見えるもの全てが現実ってことはないのあの時触れてくれた温もり光失くしては枯れていく悲しみで泣く私の涙また毒になってしまう抜け出したい私きっと'

In [22]:
# 辞書の準備
# 大量の文書を扱うときはpipeを使うと内部的にバッチ化され，効率化される
nlp = spacy.load('ja_ginza')
docs = list(nlp.pipe(removed_list))

In [27]:
# コーパスの分かち書きの作成
lyrics_wakati = []
for doc in docs:
    lyric_wakati = []
    for word in doc:
        lyric_wakati.append(word)
    lyrics_wakati.append(lyric_wakati)
print(lyrics_wakati[-2], sep='\n')  # 整形して瑠璃色金魚と花菖蒲の結果を確認

[瑠璃, 色金, 魚, は, 恋い焦がれる, 凛と, 咲き誇る, 花菖蒲, 吐き出す, 空気, は, 泡, の, 模様, 決して, あなた, の, 心, に, 届か, ない, の, はなびら, ひらひら, と, 水面, に, 落ち, て, 震える, 指先, 時間, が, 止まる, わ, 目, が, 覚め, た, 余韻, の, 余白, 外, の, 世界, は, ねえ, なんて, 眩しい, 嘘, だ, と, し, て, も, 罪深, 過ぎ, た, の, 眩暈, が, し, て, も, 心地, いい, の, は, もう, 求め, てる, から, 瑠璃, 色金, 魚, が, 見上げる, の, は, 凛と, 佇ん, だ, 花菖蒲, 私, あなた, の, よう, に, なれ, たら, もっと, 上手く, 微笑え, ます, か, 灯し, た, 明かり, は, 燃え, ない, まま, 今, も, 青く, 棚引い, て, いる, 曇っ, た, 硝子, を, 溶かす, ほど, の, 秘密, もし, か, し, て, 私, 持っ, て, ます, か, 雨, は, 空, に, 落ち, 愛すれ, ば, 消える, もの, と, 思っ, て, た, 鏡, の, 世界, に, 逆さま, に, 映っ, た, 好奇心, 湧き上がる, 思い, を, 掬い上げ, て, は, 砂糖, 漬け, に, し, て, また, 飲み込む, の, あなた, に, いつ, か, 味見, し, て, ほしい, と, 夢, を, 見, ながら, 瑠璃, 色金, 魚, が, 知ら, ない, の, は, 強く, 根, を, 張っ, た, 花菖蒲, 目, の, 前, に, 見える, もの, 全て, が, 現実, って, こと, は, ない, の, あの, 時, 触れ, て, くれ, た, 温もり, 光, 失くし, て, は, 枯れ, て, いく, 悲しみ, で, 泣く, 私, の, 涙, また, 毒, に, なっ, て, しまう, 抜け出し, たい, 瑠璃, 色金, 魚, が, 見上げる, の, は, 凛と, 佇ん, だ, 花菖蒲, 私, あなた, の, よう, に, なれ, たら, もっと, 美しく, 咲き誇れ, ます, か, 瑠璃, 色金, 魚, が, 知ら, ない, の, は, 強く, 根, を, 張っ, た, 花菖蒲, 目

In [40]:
# 分かち書きからコーパスの一意な単語のリストを作成
flatten = 
# words = sorted(list(set(lyrics)))

344